# Dagstuhl ChoirSet: Parsing and Visualization Demo

### Table of Contents
1. [DCS Content Parser](#one)
2. [Multitrack Audio Playback](#two)
3. [Beat Annotations](#three)
4. [Time-Aligned Score Representation](#four)
5. [Fundamental Frequency Trajectories](#five)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pywebaudioplayer import pywebaudioplayer as pwa
import IPython.display as ipd
import pandas as pd
import librosa
import pretty_midi

import os
import glob

import DCStoolbox as dcst

### 1. DCS Content Parser <a class="anchor" id="one"></a>

In [2]:
# Settings
DCS_path = './DagstuhlChoirSet_V1.0/'
song_id = 'DCS_LI'
setting = 'QuartetA'
take = 'Take04'
part = '*'
mic = '*'

# generate table of available audio and annotation files
DCS_table = dcst.DCS_content_parser(DCS_path, song_id, setting, take, part, mic)
DCS_table

Error: Invalid path to Dagstuhl Choir Set!

### 2. Multitrack Audio Playback<a class="anchor" id="two"></a>

**---This player is only supported in Google Chrome---**

In [ ]:
# using pywebaudioplayer by Johan Pauwels
tracks = []
for i in range(DCS_table.shape[0]):
    tracks.append({'title': '%s (%s)' %(DCS_table.iloc[i]['Section'], DCS_table.iloc[i]['Microphone']), 
                   'path': DCS_table.iloc[i]['Audio'][2:], 'mimetype': 'audio/wav'})
    
# Display trackswitch.js
ts1 = pwa.trackswitch(tracks, repeat=False, globalsolo=False, onlyradiosolo=True)
ipd.HTML(ts1)

### 3. Beat Annotations<a class="anchor" id="three"></a>

In [ ]:
fn_beat = DCS_table.iloc[1, :]['Beat']
beat_anno = dcst.read_csv(fn_beat, header=None, names=['Time (sec)', 'Measure.RelativeBeatPosition'])
beat_anno

The last row of the beat annotation includes the end of the audio file in seconds and the end of the last measure encoded as .999.

In [ ]:
# sonify beat annotation using librosa
fn_mix = DCS_table.iloc[1, :]['Audio']
x_mix, Fs = librosa.load(fn_mix)
beat_soni = librosa.clicks(beat_anno.iloc[:, 0].values, sr=Fs, click_freq=1000, length=len(x_mix))
ipd.Audio(x_mix+beat_soni, rate=Fs)

### 4. Time-Aligned Score Representation<a class="anchor" id="four"></a>

In [ ]:
# load tuple of file names
fns_score = DCS_table.iloc[1, :]['ScoreRepr']
fns_score

In [ ]:
# score representation for soprano
dcst.read_csv(fns_score[0], header=None, names=['Onset (sec)', 'Offset (sec)', 'MIDIPitch'])

In [ ]:
# sonify aligned score representation
x_soni_score = dcst.sonify_score_representation_with_sinusoidals(fns_score, Fs, len(x_mix))
ipd.Audio(np.vstack((x_mix.reshape(1, -1), x_soni_score.reshape(1, -1))), rate=Fs)

### 5. Fundamental Frequency Trajectories<a class="anchor" id="five"></a>

In [ ]:
fn_f0 = DCS_table.iloc[4, :]['F0CREPE']
f0 = dcst.read_csv(fn_f0, header=None, names=['Time (sec)', 'F0 (Hz)', 'Confidence'])
f0

In [ ]:
# stereo sonification
x_lrx, Fs = librosa.load(DCS_table.iloc[4, :]['Audio'])
x_f0 = dcst.sonify_trajectory_with_sinusoid(f0.iloc[:, 0:2].values, len(x_lrx), Fs=Fs, amplitude=0.3, smooth_len=11)
x_soni_stereo = np.vstack((x_lrx.reshape(1,-1), x_f0.reshape(1,-1)))
ipd.Audio(x_soni_stereo, rate=Fs)